In [1]:
##Importing Library
import sqlite3

In [3]:
conn = sqlite3.connect('LibraryDatabase.db')

In [5]:
cursor = conn.cursor()

In [33]:
###Creating Table Users
cursor.execute("""CREATE TABLE Users(
    id INTEGER PRIMARY KEY NOT NULL,
    full_name TEXT,
    enabled TEXT CHECK (enabled IN ('f', 't')),
    last_login DATETIME)
""")

conn.commit()

In [21]:
###Creating Table Addresses
cursor.execute("""CREATE TABLE Addresses(
    user_id INTEGER PRIMARY KEY NOT NULL,
    street TEXT,
    city TEXT,
    state TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(id))
""")

conn.commit()

In [23]:
###Creating Table Books
cursor.execute("""CREATE TABLE Books(
    id INTEGER PRIMARY KEY NOT NULL,
    title TEXT,
    author TEXT,
    published_date DATETIME,
    isbn CHAR(13) UNIQUE)
""")

conn.commit()

In [25]:
###Creating Table Checkouts
cursor.execute("""CREATE TABLE Checkouts(
    id INTEGER PRIMARY KEY NOT NULL,
    user_id INTEGER, book_id INTEGER,
    checkout_date DATETIME,
    return_date DATETIME,
    FOREIGN KEY (user_id) REFERENCES Users(id),
    FOREIGN KEY (book_id) REFERENCES Books(id))
""")

conn.commit()

In [27]:
###Creating Table Reviews
cursor.execute("""CREATE TABLE Reviews(
    id INTEGER PRIMARY KEY NOT NULL,
    book_id INTEGER, reviewer_name TEXT,
    content TEXT,
    rating INTEGER CHECK (rating IN (1, 2, 3, 4, 5)),
    published_date DATETIME,
    FOREIGN KEY (book_id) REFERENCES Books(id));
""")

conn.commit()

In [35]:
###Inserting Data in Users table
cursor.execute("""Insert into Users (full_name, enabled, last_login) values ('John Smith','f', '2017-10-25 10:26:10')""")
cursor.execute("""Insert into Users (full_name, enabled, last_login) values ('Alice Walker','t', '2017-10-25 10:26:50')""")
cursor.execute("""Insert into Users (full_name, enabled, last_login) values ('Harry Potter','t','2017-10-25 10:26:50')""")
cursor.execute("""Insert into Users (full_name, enabled, last_login) values ('Jane Smith', 't','2017-10-25 19:36:43')""")

conn.commit()

In [39]:
###User Table
cursor.execute("Select * from Users")
print(cursor.fetchall())

[(1, 'John Smith', 'f', '2017-10-25 10:26:10'), (2, 'Alice Walker', 't', '2017-10-25 10:26:50'), (3, 'Harry Potter', 't', '2017-10-25 10:26:50'), (4, 'Jane Smith', 't', '2017-10-25 19:36:43')]


In [41]:
###Inserting Data in addresses
cursor.execute("""Insert into addresses (street, city, state) values ('1 Market Street', 'San Francisco', 'CA')""")
cursor.execute("""Insert into addresses (street, city, state) values ('2 Elm Street', 'San Francisco', 'CA')""")
cursor.execute("""Insert into addresses (street, city, state) values ('3 Main Street', 'Boston', 'MA')""")

conn.commit()

In [43]:
###Addresses Table
cursor.execute("Select * from addresses")
print(cursor.fetchall())

[(1, '1 Market Street', 'San Francisco', 'CA'), (2, '2 Elm Street', 'San Francisco', 'CA'), (3, '3 Main Street', 'Boston', 'MA')]


In [45]:
###Inserting Data in books
cursor.execute("""Insert into books (title, author, published_date, isbn) values ('My First SQL book', 'Mary Parker', '2012-02-22 12:08:17','981483029127')""")
cursor.execute("""Insert into books (title, author, published_date, isbn) values ('My Second SQL book', 'John Mayer', '1972-07-03 09:22:45','857300923713')""")
cursor.execute("""Insert into books (title, author, published_date, isbn) values ('My Third SQL book', 'Cary Flint', '2015-10-18 14:05:42','523120967812')""")

conn.commit()

In [47]:
###Books Table
cursor.execute("Select * from books")
print(cursor.fetchall())

[(1, 'My First SQL book', 'Mary Parker', '2012-02-22 12:08:17', '981483029127'), (2, 'My Second SQL book', 'John Mayer', '1972-07-03 09:22:45', '857300923713'), (3, 'My Third SQL book', 'Cary Flint', '2015-10-18 14:05:42', '523120967812')]


In [49]:
###Inserting Data in reviews table
cursor.execute("""Insert into reviews (book_id, reviewer_name, content, rating, published_date) values (1,'John Smith', 'My first review',4,'2017-12-10')""")
cursor.execute("""Insert into reviews (book_id, reviewer_name, content, rating, published_date) values (2,'John Smith', 'My second review',4,'2017-10-13')""")
cursor.execute("""Insert into reviews (book_id, reviewer_name, content, rating, published_date) values (2,'Alice Walker', 'Another review',1,'2017-10-22')""")

conn.commit()

In [169]:
###Reviews Table
cursor.execute("Select * from reviews")
print(cursor.fetchall())

[(1, 1, 'John Smith', 'My first review', 4, '2017-12-10'), (2, 2, 'John Smith', 'My second review', 4, '2017-10-13'), (3, 2, 'Alice Walker', 'Another review', 1, '2017-10-22')]


In [95]:
###Trigger (Creating trigger, Creating Log Table that collects Timestamp whenever any checkout is made.)

##Creating Log Table

cursor.execute("""CREATE TABLE Logs(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    checkout_id INTEGER,
    LogTimestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (checkout_id) REFERENCES Checkouts(id))
""")

###Creating Trigger

cursor.execute("""CREATE TRIGGER AfterCheckoutInsert AFTER INSERT 
    ON Checkouts 
        BEGIN INSERT INTO Logs (checkout_id) VALUES (New.id); 
    END;
""")

conn.commit()

In [125]:
###Inserting data in checkouts then checking Log table for timestamps for each checkout.

cursor.execute("Insert into checkouts (user_id, book_id, checkout_date) values (1,1, '2017-10-15 14:43:18')")
cursor.execute("Insert into checkouts (user_id, book_id, checkout_date, return_date) values (1,2, '2017-10-05 16:22:44', '2017-10-13 13:05:12')")
cursor.execute("Insert into checkouts (user_id, book_id, checkout_date, return_date) values (2,2, '2017-10-15 11:11:24', '2017-10-22 17:42:10')")
cursor.execute("Insert into checkouts (user_id, book_id, checkout_date) values (4,3, '2017-10-15 09:27:07')")

conn.commit()

###Checkout Table
cursor.execute("Select * from checkouts")
print(cursor.fetchall())

[(1, 1, 1, '2017-10-15 14:43:18', None), (2, 1, 2, '2017-10-05 16:22:44', '2017-10-13 13:05:12'), (3, 2, 2, '2017-10-15 11:11:24', '2017-10-22 17:42:10'), (4, 4, 3, '2017-10-15 09:27:07', None)]


In [127]:
###Checking Log Table for timestamps
cursor.execute("Select * from Logs")
print(cursor.fetchall())


[(1, 1, '2024-11-04 00:27:33'), (2, 2, '2024-11-04 00:27:33'), (3, 3, '2024-11-04 00:27:33'), (4, 4, '2024-11-04 00:27:33')]


In [173]:
##Closing Connection
conn.close()

In [107]:
#SQL Queries to Pandas using read_sql_query (Importing Pandas).

import pandas as pd

In [111]:
#1

Query_1 = pd.read_sql_query("""
            SELECT Books.title, Books.author, Books.isbn 
            FROM Checkouts 
            JOIN Books ON Checkouts.book_id = Books.id 
            JOIN Users ON Checkouts.user_id = Users.id 
            WHERE Users.full_name = 'John Smith';
            """,conn)

Query_1

,title,author,isbn
0,My First SQL book,Mary Parker,981483029127
1,My Second SQL book,John Mayer,857300923713


In [117]:
#2

Query_2 = pd.read_sql_query("""
            SELECT Reviews.reviewer_name
            FROM Reviews 
            JOIN Books ON Reviews.book_id = Books.id 
            WHERE Books.title = 'My Third SQL book';
            """,conn)
Query_2

,reviewer_name


In [129]:
#3

Query_3 = pd.read_sql_query("""
            SELECT Users.full_name 
            FROM Users 
            LEFT JOIN Checkouts ON Users.id = Checkouts.user_id 
            WHERE Checkouts.user_id IS NULL;
            """,conn)
Query_3

,full_name
0,Harry Potter


In [133]:
###Queries Using Pandas Dataframes.

###Creating Dataframes for required tables.
books_df = pd.read_sql_query("SELECT * FROM Books;", conn)
users_df = pd.read_sql_query("SELECT * FROM Users;", conn)
checkouts_df = pd.read_sql_query("SELECT * FROM Checkouts;", conn)
reviews_df = pd.read_sql_query("SELECT * FROM Reviews;", conn)

In [151]:
#1

Query_1_df = checkouts_df.merge(books_df, left_on="book_id", right_on='id', how='inner') \
                     .merge(users_df, left_on="user_id", right_on='id', how='inner')
Query_1_df = Query_1_df[Query_1_df['full_name'] == 'John Smith'][['title', 'author', 'isbn']]
print("Books checked out by John Smith:")
print(Query_1_df)

Books checked out by John Smith:
                title       author          isbn
0   My First SQL book  Mary Parker  981483029127
1  My Second SQL book   John Mayer  857300923713


In [153]:
#2

Query_2_df = reviews_df.merge(books_df, left_on="book_id", right_on='id', how='inner')
Query_2_df = Query_2_df[Query_2_df['title'] == 'My Third SQL book'][['reviewer_name']]
print("All Reviewers for the book 'My Third SQL book':")
print(Query_2_df)

All Reviewers for the book 'My Third SQL book':
Empty DataFrame
Columns: [reviewer_name]
Index: []


In [165]:
#3

Query_3_df = users_df.merge(checkouts_df, left_on="id", right_on='user_id', how="left")
Query_3_df = Query_3_df[Query_3_df['user_id'].isna()][['full_name']]
print("Users with no books checked out are:")
print(Query_3_df)

Users with no books checked out are:
      full_name
3  Harry Potter


In [167]:
###SQL query showing all records from Log Table
cursor.execute("Select * from Logs")
print(cursor.fetchall())


[(1, 1, '2024-11-04 00:27:33'), (2, 2, '2024-11-04 00:27:33'), (3, 3, '2024-11-04 00:27:33'), (4, 4, '2024-11-04 00:27:33')]
